In [1]:
import numpy as np

In [2]:
def main():

    A = np.array([[2.,0.]    # 行列Aの生成
                 ,[0.,5.]])
    L = np.linalg.cholesky(A)      # 行列AをQR分解
    # 結果を表示
    print("A=\n", A)
    print("L=\n", L)
    print("L*L^T=\n", L.dot(L.T))


In [3]:
main()

A=
 [[2. 0.]
 [0. 5.]]
L=
 [[1.41421356 0.        ]
 [0.         2.23606798]]
L*L^T=
 [[2. 0.]
 [0. 5.]]


In [327]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision
from torch.autograd import Variable
import itertools
from utils import *

# eps = np.array([1.e-8])
eps = torch.autograd.Variable(torch.FloatTensor([1.e-8]), requires_grad=False)
print(eps)


class Cholesky(torch.autograd.Function):
    def forward(ctx, a):
        l = torch.cholesky(a, upper=False)
#         l = torch.potrf(a, False)
        ctx.save_for_backward(l)
        return l
    def backward(ctx, grad_output):
        l, = ctx.saved_variables
        linv = l.inverse()
        inner = torch.tril(torch.mm(l.t(), grad_output)) * torch.tril(
            1.0 - Variable(l.data.new(l.size(1)).fill_(0.5).diag()))
        s = torch.mm(linv.t(), torch.mm(inner, linv))
        return s

    
######################################
#   Compression Network
######################################
class CompressionNetworkArrhythmia(nn.Module):
    """Defines a compression network for the Arrhythmia dataset as described in
    the paper."""
    def __init__(self):
        super().__init__()

        self.encoder = nn.Sequential(nn.Linear(32, 10),
                                     nn.Tanh(),
                                     nn.Linear(10, 2))
        self.decoder = nn.Sequential(nn.Linear(2, 10),
                                     nn.Tanh(),
                                     nn.Linear(10, 32))

        self._reconstruction_loss = nn.MSELoss()

    def forward(self, input):
        out = self.encoder(input)
        out = self.decoder(out)

        return out

    def encode(self,  input):
        return self.encoder(input)

    def decode(self, input):
        return self.decoder(input)

    def reconstruction_loss(self, input, target):
        target_hat = self(input)
        return self._reconstruction_loss(target_hat, target)

######################################
#   Estimation Network
######################################
class EstimationNetworkArrhythmia(nn.Module):
    """Defines a estimation network for the Arrhythmia dataset as described in
    the paper."""
    def __init__(self, n_gmm=2, latent_dim=4):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(4, 10),
                                 nn.Tanh(),
                                 nn.Dropout(p=0.5),
                                 nn.Linear(10, 2),
                                 nn.Softmax(dim=1))

    def forward(self, input):
        return self.net(input)

    

######################################
#   DAGMM
######################################
class DAGMM(nn.Module):
    """Residual Block."""
    def __init__(self, n_gmm = 2, latent_dim=4):
        super(DAGMM, self).__init__()
        
        self.eps = torch.autograd.Variable(torch.FloatTensor([1.e-8]), requires_grad=False)
        compressor = CompressionNetworkArrhythmia()
        estimator = EstimationNetworkArrhythmia(n_gmm, latent_dim)
        
#         layers = []
#         layers += [nn.Linear(118,60)]
#         layers += [nn.Tanh()]        
#         layers += [nn.Linear(60,30)]
#         layers += [nn.Tanh()]        
#         layers += [nn.Linear(30,10)]
#         layers += [nn.Tanh()]        
#         layers += [nn.Linear(10,1)]

        self.encoder = compressor.encoder


#         layers = []
#         layers += [nn.Linear(1,10)]
#         layers += [nn.Tanh()]        
#         layers += [nn.Linear(10,30)]
#         layers += [nn.Tanh()]        
#         layers += [nn.Linear(30,60)]
#         layers += [nn.Tanh()]        
#         layers += [nn.Linear(60,118)]

        self.decoder = compressor.decoder
    
#         layers = []
#         layers += [nn.Linear(latent_dim,10)]
#         layers += [nn.Tanh()]        
#         layers += [nn.Dropout(p=0.5)]        
#         layers += [nn.Linear(10,n_gmm)]
#         layers += [nn.Softmax(dim=1)]


        self.estimation = estimator.net

        self.register_buffer("phi", torch.zeros(n_gmm))
        self.register_buffer("mu", torch.zeros(n_gmm,latent_dim))
        self.register_buffer("cov", torch.zeros(n_gmm,latent_dim,latent_dim))
   
    def relative_euclidean_distance(self, x1, x2, eps):
        """
        x1 and x2 are assumed to be Variables or Tensors.
        They have shape [batch_size, dimension_embedding]
        相対ユークリッド距離の計算、数式は論文を参照
        """

#         # numpy array で計算
#         num = np.linalg.norm(x1 -x2, ord=2)
#         denom = np.linalg.norm(x1, ord=1)
#         return num / np.maximum(denom, eps)
   
        # torch で計算
        print(f'x1 : {x1}')
        print(f'x22 : {x2}')
        num = torch.norm(x1 - x2, p=2, dim=1)  # dim [batch_size]
        denom = torch.norm(x1, p=2, dim=1)  # dim [batch_size]
        print(f'num : {num}')
        print(f'denom : {denom}')
        return num / torch.max(denom, eps)

    def cosine_similarity(self, x1, x2, eps):
        """
        x1 and x2 are assumed to be Variables or Tensors.
        They have shape [batch_size, dimension_embedding
        コサイン類似度の計算、数式は論文を参照
        """
        
#         # numpy arrayで計算
#         dot_prod = np.sum(x1 * x2)
#         dist_x1 = np.linalg.norm(x1, ord=2)
#         dist_x2 = np.linalg.norm(x2, ord=2)
#         return dot_prod / np.maximum(dist_x1 * dist_x2, eps)
        
        # torch で計算
        dot_prod = torch.sum(x1 * x2, dim=1)  # dim [batch_size]
        dist_x1 = torch.norm(x1, p=2, dim=1)  # dim [batch_size]
        dist_x2 = torch.norm(x2, p=2, dim=1)  # dim [batch_size]
        return dot_prod / torch.max(dist_x1*dist_x2, eps)


#     def relative_euclidean_distance(self, a, b):
#         return (a-b).norm(2, dim=1) / a.norm(2, dim=1)

    def forward(self, x):

        enc = self.encoder(x)

        dec = self.decoder(enc)

#         rec_cosine = self.cosine_similarity(x, dec, self.eps)
        rec_cosine = F.cosine_similarity(x, dec)
        rec_euclidean = self.relative_euclidean_distance(x, dec, self.eps)

        z = torch.cat([enc, rec_euclidean.unsqueeze(-1), rec_cosine.unsqueeze(-1)], dim=1)

        gamma = self.estimation(z)

        return enc, dec, z, gamma

    def compute_gmm_params(self, z, gamma):
        N = gamma.size(0)
        # K
        sum_gamma = torch.sum(gamma, dim=0)

        # K
        phi = (sum_gamma / N)

        self.phi = phi.data

 
        # K x D
        mu = torch.sum(gamma.unsqueeze(-1) * z.unsqueeze(1), dim=0) / sum_gamma.unsqueeze(-1)
        self.mu = mu.data
        # z = N x D
        # mu = K x D
        # gamma N x K

        # z_mu = N x K x D
        z_mu = (z.unsqueeze(1)- mu.unsqueeze(0))

        # z_mu_outer = N x K x D x D
        z_mu_outer = z_mu.unsqueeze(-1) * z_mu.unsqueeze(-2)

        # K x D x D
        cov = torch.sum(gamma.unsqueeze(-1).unsqueeze(-1) * z_mu_outer, dim = 0) / sum_gamma.unsqueeze(-1).unsqueeze(-1)
        self.cov = cov.data

        return phi, mu, cov
        
    def compute_energy(self, z, phi=None, mu=None, cov=None, size_average=True):
        """
        return :
            sample_energy
                E(z) = -log(∑ φ exp(-1/2 (z-μ)T ∑inv (z-μ)) / sqrt(|2π∑|))
            cov_diag
                共分散対角行列
                
        """
        if phi is None:
            phi = to_var(self.phi)
        if mu is None:
            mu = to_var(self.mu)
        if cov is None:
            cov = to_var(self.cov)

        k, D, _ = cov.size()

        
        z_mu = (z.unsqueeze(1)- mu.unsqueeze(0))

        
        cov_inverse = []
        det_cov = []
        cov_diag = 0
        eps = 1e-12
        for i in range(k):
            # K x D x D
            cov_k = cov[i] + to_var(torch.eye(D)*eps)
            cov_inverse.append(torch.inverse(cov_k).unsqueeze(0))

            #det_cov.append(np.linalg.det(cov_k.data.cpu().numpy()* (2*np.pi)))
            det_cov.append((Cholesky.apply(cov_k.cpu() * (2*np.pi)).diag().prod()).unsqueeze(0))
            cov_diag = cov_diag + torch.sum(1 / cov_k.diag())

        # K x D x D
        cov_inverse = torch.cat(cov_inverse, dim=0)
        # K
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        det_cov = torch.cat(det_cov)
        det_cov = det_cov.to(device)
        #det_cov = to_var(torch.from_numpy(np.float32(np.array(det_cov))))

        # N x K
        exp_term_tmp = -0.5 * torch.sum(torch.sum(z_mu.unsqueeze(-1) * cov_inverse.unsqueeze(0), dim=-2) * z_mu, dim=-1)
        # for stability (logsumexp)
        max_val = torch.max((exp_term_tmp).clamp(min=0), dim=1, keepdim=True)[0]

        exp_term = torch.exp(exp_term_tmp - max_val)

        # sample_energy = -max_val.squeeze() - torch.log(torch.sum(phi.unsqueeze(0) * exp_term / (det_cov).unsqueeze(0), dim = 1) + eps)
#         sample_energy = -max_val.squeeze() - torch.log(torch.sum(phi.unsqueeze(0) * exp_term / (torch.sqrt(det_cov)).unsqueeze(0), dim = 1) + eps)
        sample_energy = -max_val.squeeze() - torch.log(torch.sum(phi.unsqueeze(0) * exp_term / (torch.sqrt((2*np.pi)**D * det_cov)).unsqueeze(0), dim = 1) + eps)


        if size_average:
            sample_energy = torch.mean(sample_energy)

        return sample_energy, cov_diag


    def loss_function(self, x, x_hat, z, gamma, lambda_energy, lambda_cov_diag):

        recon_error = torch.mean((x - x_hat) ** 2)

        phi, mu, cov = self.compute_gmm_params(z, gamma)

        sample_energy, cov_diag = self.compute_energy(z, phi, mu, cov)

        loss = recon_error + lambda_energy * sample_energy + lambda_cov_diag * cov_diag

        return loss, sample_energy, recon_error, cov_diag


tensor([1.0000e-08])


In [322]:
gmm = DAGMM()

In [296]:
input_np = np.random.random([5, 32])
print(input_np)
input_ts = torch.autograd.Variable(torch.from_numpy(input_np).float())
print(input_ts)

[[0.83216248 0.15188222 0.7761958  0.27129416 0.38971093 0.8161424
  0.61976807 0.02061973 0.10195588 0.71028057 0.08749903 0.21319041
  0.8045891  0.45488211 0.72080927 0.26068037 0.89419051 0.19932481
  0.78187668 0.88199391 0.66339858 0.92554494 0.12324947 0.25364563
  0.19261142 0.20012666 0.93893405 0.88703559 0.88357943 0.08238283
  0.11887701 0.12406312]
 [0.96520811 0.70356366 0.67638255 0.81723369 0.44614467 0.62861295
  0.2789295  0.71144148 0.94371077 0.75329837 0.34940377 0.38202543
  0.89730189 0.17160973 0.76649312 0.72053461 0.44027273 0.80069073
  0.32415927 0.99607569 0.77292314 0.87102227 0.91608273 0.34385432
  0.69908196 0.39244787 0.74322366 0.61411631 0.49533703 0.02329461
  0.16132873 0.1012299 ]
 [0.85246669 0.13133938 0.17539389 0.01115635 0.36012947 0.10138671
  0.57756541 0.06700133 0.47960979 0.87739291 0.9971416  0.93099248
  0.26314384 0.3852812  0.34733085 0.46975059 0.36806407 0.56743816
  0.00121871 0.5367064  0.78952665 0.70760836 0.71113395 0.71043367

# test forward

In [287]:
enc, dec, z, gamma = gmm.forward(input_ts)
print(f'enc:\n{enc}')
print(f'dec:\n{dec}')
print(f'z:\n{z}')
print(f'gamma:\n{gamma}')
print(f'\n')
print(f'enc shape:\n{enc.shape}')
print(f'dec shape:\n{dec.shape}')
print(f'z shape:\n{z.shape}')
print(f'gamma shape:\n{gamma.shape}')
print(f'\n')
print(f'enc type:\n{type(enc)}')
print(f'dec type:\n{type(dec)}')
print(f'z type:\n{type(z)}')
print(f'gamma type:\n{type(gamma)}')


enc:
tensor([[-0.1627,  0.3520],
        [-0.2137,  0.4200],
        [-0.2293,  0.0741],
        [-0.1675,  0.1354],
        [-0.2195,  0.3743]], grad_fn=<AddmmBackward>)
dec:
tensor([[ 3.2856e-01,  2.1979e-01,  3.1413e-01, -1.8229e-01,  5.0145e-01,
         -2.2579e-01, -3.5307e-01, -2.5069e-01, -2.5797e-01,  5.3137e-02,
          4.3190e-02,  6.7920e-01, -6.1335e-02,  1.4863e-02,  4.5900e-01,
         -4.4890e-01,  1.3823e-01, -4.0124e-02,  1.0513e-01, -2.3167e-01,
          2.6358e-01,  6.2020e-01,  2.3446e-01,  2.5876e-01,  3.7526e-01,
         -1.3770e-01, -3.4357e-02,  1.3643e-03,  1.0844e-01,  1.6957e-01,
          6.5526e-02,  2.4404e-01],
        [ 3.5790e-01,  1.8431e-01,  3.1972e-01, -1.6071e-01,  5.0431e-01,
         -2.7042e-01, -3.5724e-01, -2.4574e-01, -2.5089e-01,  4.5844e-02,
         -7.1206e-03,  6.6128e-01, -7.9353e-02,  5.1596e-02,  4.0523e-01,
         -4.4704e-01,  1.1339e-01, -1.3093e-02,  1.5329e-01, -2.1366e-01,
          2.6222e-01,  6.2408e-01,  2.4008e-01, 

# test compute_gmm_params

In [288]:
phi, mu, cov= gmm.compute_gmm_params(z=z, gamma=gamma)

print(f'phi:\n{phi}')
print(f'mu:\n{mu}')
print(f'cov:\n{cov}')
print(f'\n')
print(f'phi shape:\n{phi.shape}')
print(f'mu shape:\n{mu.shape}')
print(f'cov shape:\n{cov.shape}')
print(f'\n')
print(f'phi type:\n{type(phi)}')
print(f'mu type:\n{type(mu)}')
print(f'cov: type\n{type(cov)}')


phi:
tensor([0.4910, 0.5090], grad_fn=<DivBackward0>)
mu:
tensor([[-0.1988,  0.2769,  0.9971,  0.2589],
        [-0.1983,  0.2657,  0.9986,  0.2513]], grad_fn=<DivBackward0>)
cov:
tensor([[[ 7.2142e-04, -6.1808e-04, -5.8306e-04,  8.4065e-04],
         [-6.1808e-04,  1.9065e-02,  6.2524e-03, -1.1416e-02],
         [-5.8306e-04,  6.2524e-03,  3.0817e-03, -5.2522e-03],
         [ 8.4065e-04, -1.1416e-02, -5.2522e-03,  9.3904e-03]],

        [[ 8.2206e-04,  7.3147e-04, -1.5330e-04, -6.7713e-05],
         [ 7.3147e-04,  1.9510e-02,  5.1974e-03, -1.0143e-02],
         [-1.5330e-04,  5.1974e-03,  2.2896e-03, -4.0687e-03],
         [-6.7713e-05, -1.0143e-02, -4.0687e-03,  7.7273e-03]]],
       grad_fn=<DivBackward0>)


phi shape:
torch.Size([2])
mu shape:
torch.Size([2, 4])
cov shape:
torch.Size([2, 4, 4])


phi type:
<class 'torch.Tensor'>
mu type:
<class 'torch.Tensor'>
cov: type
<class 'torch.Tensor'>


# test compute_energy

In [289]:
z = to_var(z)
energy, cov_diag = gmm.compute_energy(z=z,
#                                       phi=phi, mu=mu, cov=cov
                                     )

print(f'sample energy:\n{energy}')
print(f'cov diag:\n{cov_diag}')
print(f'\n')
print(f'sample energy shape:\n{energy.shape}')
print(f'cov diag shape:\n{cov_diag.shape}')
print(f'\n')
print(f'sample energy type:\n{type(energy)}')
print(f'cov diag type:\n{type(cov_diag)}')


sample energy:
0.9110661745071411
cov diag:
3703.48828125


sample energy shape:
torch.Size([])
cov diag shape:
torch.Size([])


sample energy type:
<class 'torch.Tensor'>
cov diag type:
<class 'torch.Tensor'>


# test loss function

In [290]:
input_ts = to_var(input_ts)
dec = to_var(dec)
gamma = to_var(gamma)
loss, sample_energy, recon_error, cov_diag = gmm.loss_function(x=input_ts, x_hat=dec, z=z, gamma=gamma, lambda_energy=energy, lambda_cov_diag=cov_diag)

print(f'loss:\n{loss}')
print(f'sample energy:\n{sample_energy}')
print(f'recon error:\n{recon_error}')
print(f'cov diag:\n{cov_diag}')
print(f'\n')
print(f'loss shape:\n{loss.shape}')
print(f'sample energy shape:\n{energy.shape}')
print(f'recon error shape:\n{recon_error.shape}')
print(f'cov diag shape:\n{cov_diag.shape}')
print(f'\n')
print(f'loss type:\n{type(loss)}')
print(f'sample energy type:\n{type(energy)}')
print(f'cov diag type:\n{type(cov_diag)}')
print(f'cov diag type:\n{type(cov_diag)}')


loss:
13715826.0
sample energy:
0.9110661745071411
recon error:
0.34675270318984985
cov diag:
3703.48828125


loss shape:
torch.Size([])
sample energy shape:
torch.Size([])
recon error shape:
torch.Size([])
cov diag shape:
torch.Size([])


loss type:
<class 'torch.Tensor'>
sample energy type:
<class 'torch.Tensor'>
cov diag type:
<class 'torch.Tensor'>
cov diag type:
<class 'torch.Tensor'>


In [291]:
x1 = torch.FloatTensor(np.array([4]))
x2 = torch.FloatTensor(np.array([5]))

# x1 = np.array([4,5,6,7])
# x2 = np.array([5,6,7,8])


In [292]:
gmm.parameters

<bound method Module.parameters of DaGMM(
  (encoder): Sequential(
    (0): Linear(in_features=32, out_features=10, bias=True)
    (1): Tanh()
    (2): Linear(in_features=10, out_features=2, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=2, out_features=10, bias=True)
    (1): Tanh()
    (2): Linear(in_features=10, out_features=32, bias=True)
  )
  (estimation): Sequential(
    (0): Linear(in_features=4, out_features=10, bias=True)
    (1): Tanh()
    (2): Dropout(p=0.5)
    (3): Linear(in_features=10, out_features=2, bias=True)
    (4): Softmax()
  )
)>

In [8]:
print_network(gmm, 'DaGMM')

DaGMM
DaGMM(
  (encoder): Sequential(
    (0): Linear(in_features=118, out_features=60, bias=True)
    (1): Tanh()
    (2): Linear(in_features=60, out_features=30, bias=True)
    (3): Tanh()
    (4): Linear(in_features=30, out_features=10, bias=True)
    (5): Tanh()
    (6): Linear(in_features=10, out_features=1, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=1, out_features=10, bias=True)
    (1): Tanh()
    (2): Linear(in_features=10, out_features=30, bias=True)
    (3): Tanh()
    (4): Linear(in_features=30, out_features=60, bias=True)
    (5): Tanh()
    (6): Linear(in_features=60, out_features=118, bias=True)
  )
  (estimation): Sequential(
    (0): Linear(in_features=3, out_features=10, bias=True)
    (1): Tanh()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=10, out_features=2, bias=True)
    (4): Softmax(dim=1)
  )
)
The number of parameters: 18761


In [6]:
def print_network(model, name):
        num_params = 0
        for p in model.parameters():
            num_params += p.numel()
        print(name)
        print(model)
        print("The number of parameters: {}".format(num_params))


In [11]:
def load_pretrained_model():
        self.dagmm.load_state_dict(torch.load(os.path.join(
            self.model_save_path, '{}_dagmm.pth'.format(self.pretrained_model))))

        print("phi", self.dagmm.phi,"mu",self.dagmm.mu, "cov",self.dagmm.cov)

        print('loaded trained models (step: {})..!'.format(self.pretrained_model))


In [13]:
gmm.load_state_dict

<bound method Module.load_state_dict of DaGMM(
  (encoder): Sequential(
    (0): Linear(in_features=118, out_features=60, bias=True)
    (1): Tanh()
    (2): Linear(in_features=60, out_features=30, bias=True)
    (3): Tanh()
    (4): Linear(in_features=30, out_features=10, bias=True)
    (5): Tanh()
    (6): Linear(in_features=10, out_features=1, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=1, out_features=10, bias=True)
    (1): Tanh()
    (2): Linear(in_features=10, out_features=30, bias=True)
    (3): Tanh()
    (4): Linear(in_features=30, out_features=60, bias=True)
    (5): Tanh()
    (6): Linear(in_features=60, out_features=118, bias=True)
  )
  (estimation): Sequential(
    (0): Linear(in_features=3, out_features=10, bias=True)
    (1): Tanh()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=10, out_features=2, bias=True)
    (4): Softmax(dim=1)
  )
)>

In [15]:
gmm.zero_grad()

# data load

In [120]:
sys.path.append('../../fastd-asd-experiments/src/utils/')
sys.path.append('../../fastd-asd-experiments/src/')
sys.path.append('../../fastd-asd-experiments/')

In [121]:
import utils
import ExpUtils

In [131]:
util = ExpUtils.ExpUtils()
data_set = '001_HITACHI_fan'
anr = '0db'
feature_type = 'FASTD_standard'

dataset_path = util.find_feature_data_set(data_set, anr, feature_type)
display(dataset_path)
dataset = util.load_feature_data_set(dataset_path)
norm_train = dataset[3]
anom_train = dataset[1]
norm_test  = dataset[2]
anom_test  = dataset[0]

['/work-hmcomm/Experiment_DataSet/feature_dataset/001_HITACHI_fan/0db/FASTD_standard/csv/001_HITACHI_fan_0db_anom_test.csv',
 '/work-hmcomm/Experiment_DataSet/feature_dataset/001_HITACHI_fan/0db/FASTD_standard/csv/001_HITACHI_fan_0db_anom_train.csv',
 '/work-hmcomm/Experiment_DataSet/feature_dataset/001_HITACHI_fan/0db/FASTD_standard/csv/001_HITACHI_fan_0db_norm_test.csv',
 '/work-hmcomm/Experiment_DataSet/feature_dataset/001_HITACHI_fan/0db/FASTD_standard/csv/001_HITACHI_fan_0db_norm_train.csv']

In [133]:
# 使わないカラムの削除
drop_columns = ['wav_name', 'target', 'energy', 'mfcc_1']
norm_train = norm_train.drop(columns=drop_columns)
anom_train = anom_train.drop(columns=drop_columns)
norm_test  = norm_test.drop(columns=drop_columns)
anom_test  = anom_test.drop(columns=drop_columns)

In [136]:
norm_train.shape

(31551, 32)

In [178]:
class DataSet(torch.utils.data.Dataset):
    def __init__(self, ):
        # TODO
        # 1. ファイルパスまたはファイル名のリストを初期化
        util = ExpUtils.ExpUtils()
        data_set = '001_HITACHI_fan'
        anr = '0db'
        feature_type = 'FASTD_standard'
        dataset_path = util.find_feature_data_set(data_set, anr, feature_type)
        self.dataset = util.load_feature_data_set(dataset_path)
        self.drop_columns = ['wav_name', 'target', 'energy', 'mfcc_1']
    
    def __getitem__(self, index):
        
        # TODO
        # 1. ファイルから1つのデータを読み込む
        # 2. データの前処理
        # 3. データのペアを返す
        norm_train = self.dataset[3].drop(columns=self.drop_columns)
        anom_train = self.dataset[1].drop(columns=self.drop_columns)
        norm_test  = self.dataset[2].drop(columns=self.drop_columns)
        anom_test  = self.dataset[0].drop(columns=self.drop_columns)
        
        return norm_train.iloc[index,:].values, anom_train.iloc[index,:].values, norm_test.iloc[index,:].values, anom_test.iloc[index,:].values
        
    def __len__(self):
        # 少なくとも０を設定する
        return len(self.dataset[0])

    

In [180]:
data_set = DataSet()

In [181]:
norm_train, _, _, _  = data_set[0]

In [182]:
norm_train

array([ 1.44643080e-01,  3.30201341e+00,  2.42535491e-01,  2.25260308e-01,
        1.37212106e+00,  2.83849890e-04,  2.69000000e-01,  1.15565540e+00,
       -9.47507137e-02,  3.98728772e-02, -1.98528710e-01,  2.49706698e-01,
        1.62674242e-01,  8.65621306e-02,  6.64090488e-02,  8.16060156e-02,
       -1.44304370e-01,  1.81302740e-02, -3.87194224e-02,  1.57128081e-03,
        7.09372068e-03,  1.86572910e-02,  4.65035238e-03,  1.24156651e-01,
        3.21724920e-03,  3.19258434e-03,  2.34249430e-03,  1.83047468e-03,
        6.16974642e-03,  2.67911273e-03,  1.21491189e-03,  3.33033310e-02])

In [201]:
data_loader = torch.utils.data.DataLoader(dataset=data_set, batch_size=1000, shuffle=True)

In [230]:
class Autoencoder(nn.Module):
    
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(32, 20),
            nn.ReLU(True),
            nn.Linear(20, 10),
            nn.ReLU(True),
            nn.Linear(10, 5)
        ) 
        self.decoder = nn.Sequential(
            nn.Linear(5, 10),
            nn.ReLU(True),
            nn.Linear(10, 20),
            nn.ReLU(True),
            nn.Linear(20, 32),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

model = Autoencoder()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

Autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=32, out_features=20, bias=True)
    (1): ReLU(inplace)
    (2): Linear(in_features=20, out_features=10, bias=True)
    (3): ReLU(inplace)
    (4): Linear(in_features=10, out_features=5, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=5, out_features=10, bias=True)
    (1): ReLU(inplace)
    (2): Linear(in_features=10, out_features=20, bias=True)
    (3): ReLU(inplace)
    (4): Linear(in_features=20, out_features=32, bias=True)
    (5): Tanh()
  )
)

In [216]:
epochs = 4
for epoch in range(epochs):
    print(f'epoch : {epoch}/{epochs}')
    for i in data_loader:
        norm = i[0]
        print(norm.shape)
#         norm = torch.FloatTensor(norm)
        norm.float()
        print(type(norm))
    break

epoch : 0/4
torch.Size([1000, 32])
<class 'torch.Tensor'>
torch.Size([1000, 32])
<class 'torch.Tensor'>
torch.Size([1000, 32])
<class 'torch.Tensor'>
torch.Size([198, 32])
<class 'torch.Tensor'>


In [232]:
learning_rate = 0.001
num_epochs = 5

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=learning_rate,
                             weight_decay=1e-5)

loss_list = []

for epoch in range(num_epochs):
    for data in data_loader:
        x_norm = data[0].float()
#         x = img.view(img.size(0), -1)
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        x_norm = x_norm.to(device) 
        x_norm = Variable(x_norm)
        xhat_norm = model(x_norm)
        # 再構成と入力との間でlossを計算
        loss = criterion(xhat_norm, x_norm)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # logging
        loss_list.append(loss)
    
    print('epoch [{}/{}], loss: {:.4f}'.format(
        epoch + 1,
        num_epochs,
        loss))



epoch [1/5], loss: 0.4927
epoch [2/5], loss: 0.4858
epoch [3/5], loss: 0.4811
epoch [4/5], loss: 0.4701
epoch [5/5], loss: 0.4675


# 訓練ループ

In [328]:
class Solver(object):
    def __init__(self, data_loader):
        self.num_epoch = 10000
        self.num_batch = 128
        self.lr = 0.001 
        self.data_loader = data_loader
        
        # build model
        self.build_model()
    
    def build_model(self):
        # Define model
        self.dagmm = DAGMM()

        # Optimizers
        self.optimizer = torch.optim.Adam(self.dagmm.parameters(), lr=self.lr)

        # Print network
        num_params = 0
        for p in self.dagmm.parameters():
            num_params += p.numel()
        print(self.dagmm)
        print(f'model:name : DaGMM')
        print(f'parameterの数 : {num_params}')
        
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.dagmm = self.dagmm.to(self.device)

    def reset_grad(self):
        self.dagmm.zero_grad()
        
    def to_var(self, x, volatile=False):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        x = x.to(self.device)
        return Variable(x, volatile=volatile)
    
    def train(self):
        start = 0
        
        # start training 
        iter_ctr = 0
        for e in range(start, self.num_epoch):
            for i , input_data in enumerate(self.data_loader): 
                input_data = input_data[0].float()
                input_data = self.to_var(input_data)
                print(input_data)
                
                total_loss, sample_energy, cov_diag = self.dagmm_step(input_data)
                
                loss = {}
                loss['total_loss'] = total_loss.data.item()
                loss['sample_energy'] = sample_energy.item()
                loss['recon_error'] = recon_error.item()
                loss['cov_diag'] = cov_diag.item()
                
                print('epoch [{}/{}], loss: {:.4f}'.format(
                        epoch + 1,
                        num_epochs,
                        loss))
                
    def dagmm_step(self, input_data):
        self.dagmm.train()
        enc, dec, z, gamma = self.dagmm(input_data)
        
        total_loss, sample_energy, recon_error, cov_diag = self.dagmm.loss_function(input_data, dec, z, gamma, self.lamda_energy)
        self.rest_grad()
        total_loss.backward()
        self.nn.utils.clip_grad_norm_(self.dagmm.parameteers(), 5)
        self.optimizer.step()
        
        return total_loss, sample_energy, recon_error, cov_diag

In [329]:
solver = Solver(data_loader)

DAGMM(
  (encoder): Sequential(
    (0): Linear(in_features=32, out_features=10, bias=True)
    (1): Tanh()
    (2): Linear(in_features=10, out_features=2, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=2, out_features=10, bias=True)
    (1): Tanh()
    (2): Linear(in_features=10, out_features=32, bias=True)
  )
  (estimation): Sequential(
    (0): Linear(in_features=4, out_features=10, bias=True)
    (1): Tanh()
    (2): Dropout(p=0.5)
    (3): Linear(in_features=10, out_features=2, bias=True)
    (4): Softmax()
  )
)
model:name : DaGMM
parameterの数 : 806


In [330]:
solver.train()

tensor([[5.8507e-02, 3.2606e+00, 1.3989e-01,  ..., 1.0499e-02, 3.6066e-03,
         2.9209e-02],
        [8.0760e-02, 3.2631e+00, 2.4379e-01,  ..., 4.0047e-03, 1.7704e-03,
         3.9375e-02],
        [5.5507e-02, 3.2665e+00, 1.4651e-01,  ..., 2.1727e-03, 1.5564e-03,
         3.7637e-02],
        ...,
        [1.1339e-01, 3.2938e+00, 2.2231e-01,  ..., 3.6866e-03, 4.2515e-03,
         4.6581e-02],
        [8.3760e-02, 3.2500e+00, 1.8968e-01,  ..., 1.2751e-02, 1.7834e-03,
         5.1494e-02],
        [9.8762e-02, 3.3075e+00, 2.0993e-01,  ..., 8.0711e-03, 4.0048e-03,
         1.6999e-02]], device='cuda:0')
x1 : tensor([[5.8507e-02, 3.2606e+00, 1.3989e-01,  ..., 1.0499e-02, 3.6066e-03,
         2.9209e-02],
        [8.0760e-02, 3.2631e+00, 2.4379e-01,  ..., 4.0047e-03, 1.7704e-03,
         3.9375e-02],
        [5.5507e-02, 3.2665e+00, 1.4651e-01,  ..., 2.1727e-03, 1.5564e-03,
         3.7637e-02],
        ...,
        [1.1339e-01, 3.2938e+00, 2.2231e-01,  ..., 3.6866e-03, 4.2515e-03,
   

RuntimeError: Expected object of backend CUDA but got backend CPU for argument #2 'other'

In [ ]:
dagmm = DaGMM()

In [ ]:
dagmm.loss_function()

In [80]:
solver = Solver()

In [81]:
solver.build_model()

DaGMM(
  (encoder): Sequential(
    (0): Linear(in_features=274, out_features=10, bias=True)
    (1): Tanh()
    (2): Linear(in_features=10, out_features=2, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=2, out_features=10, bias=True)
    (1): Tanh()
    (2): Linear(in_features=10, out_features=274, bias=True)
  )
  (estimation): Sequential(
    (0): Linear(in_features=4, out_features=10, bias=True)
    (1): Tanh()
    (2): Dropout(p=0.5)
    (3): Linear(in_features=10, out_features=2, bias=True)
    (4): Softmax()
  )
)
model:name : DaGMM
parameterの数 : 5888


In [82]:
solver.reset_grad()

In [83]:
input_ts

tensor([[0.5525, 0.0565, 0.2837,  ..., 0.2132, 0.7036, 0.2365],
        [0.3383, 0.3235, 0.2650,  ..., 0.5296, 0.7910, 0.5985],
        [0.2078, 0.6065, 0.8221,  ..., 0.7976, 0.5674, 0.0793],
        [0.3996, 0.3767, 0.5216,  ..., 0.1398, 0.5509, 0.0792],
        [0.8228, 0.2886, 0.9797,  ..., 0.0307, 0.1025, 0.1172]])

In [84]:
test = solver.to_cuda(input_ts)

In [85]:
test

tensor([[0.5525, 0.0565, 0.2837,  ..., 0.2132, 0.7036, 0.2365],
        [0.3383, 0.3235, 0.2650,  ..., 0.5296, 0.7910, 0.5985],
        [0.2078, 0.6065, 0.8221,  ..., 0.7976, 0.5674, 0.0793],
        [0.3996, 0.3767, 0.5216,  ..., 0.1398, 0.5509, 0.0792],
        [0.8228, 0.2886, 0.9797,  ..., 0.0307, 0.1025, 0.1172]],
       device='cuda:0')